# 1 - Introduction et problématique

petite intro des familles toi meme tu sais test:::

# 2 - Récupération et traitement des données

Cette section a pour objectif de constituer la base de donnée qui sera assujettie à l'ensemble de nos analyses.
Pour commencer, nous pouvons nous rendre sur le site officiel de la IMDB : https://www.imdb.com/
Sur le haut de la page, il est possible d'effectuer une recherche personnalisée afin d'obtenir une liste très étendue de films :
<img src= "images/image_1.png" />

Une fois la recherche lancée, le site nous redirige vers le début de la liste souhaitée :
<img src="images/image_2.png" />

On peut observer plusieures informations : 447 167 films sont à notre disposition, avec 250 films par page. De plus pour chaque film, on a accès à plusieurs informations : son titre, année de sortie, durée, genres principaux, note du public, nombre de votes, metascore, directeurs, acteurs. 
L'objectif est de constituer notre base de données en regroupant dans un dataframe toutes ces informations pour chaque film. 

Et rien de plus essentiel pour effectuer cette tâche que de mettre en place une belle opération de web scraping ! Moteur, et ... Action !

## 2.1 - Création d'une base IMDB web scrapée

Avant toute chose, il est important d'importer les modules qui nous seront indispensables :

In [35]:
import pandas as pd
import sys
sys.path.append('./modules')
from scraping_cleaning import imdb_scraping

In [36]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Ensuite on crée les tableaux vides pour les remplir des informations par la suite :

In [37]:
results = imdb_scraping.init_results()

Ensuite, on prend l'url de la toute première page (début de la liste des films à scrapper), puis on l'insère dans une liste. Ceci nous permettra de lancer l'instruction de la première page à scrapper au tout début de la boucle. Puis dans la boule on récoltera les liens des pages suivantes qu'on stockera également dans cette liste :

In [38]:
#URL à mettre à jour à chaque session de scraping
url = "https://www.imdb.com/search/title/?title_type=feature&adult=include&count=250&after=Wzc0MjQxMywidHQyMjk0NjY1IiwyNzUwMDFd&ref_=adv_nxt"
list_url = [url]

In [39]:
df = imdb_scraping.fill_results(3,list_url,results)

Comme il y a près de 450000 films à web scrapper et qu'il y a 250 films par pages, nous avons environ N = 450000/250 = 1800 pages à web scrapper.

Comme le nombre de films à scrapper est conséquent, la boucle de web scraping est également très longue à s'executer. Par motif de précaution, nous avons décidé de fragmenter la boucle : plutôt que de lancer une seule boucle de N = 1800 pages à scrapper, nous lancerons 6 boucles de 300 pages à web scrap. A la fin de chaque boucle, nous prendrons le dernier url scrappé, qui sera l'url de départ de la boucle qui suivra.
A la fin de chaque boucle, nous enregistrerons chaque lot de données dans une base pickle et une autre base au format csv. A la fin, nous n'aurons plus qu'à importer nos 6 bases csv, puis effectuer une jointure, et nous obtiendrons notre dataframe d'environ 450 000 observations

A titre de démonstration, nous nous contenterons de lancer ici uniquement une boucle pour web scraper 1 page, pour montrer que la boucle marche bien sans avoir à attendre une éternité.

Une fois la boucle terminée, on peut simplement s'assurer que tout s'est bien déroulé en affichant la dataframe : 

In [40]:
df

,tconst,title,year,runtime,genres,metascore,rate,votes,certificate,director,casting
0,[tt0064803],Pesn o Manshuk,(1970),78 min,"\nDrama, War",NA,4.3,14,NA,Mazhit Begalin,"[Natalya Arinbasarova, Nikita Mikhalkov, Ivan ..."
1,[tt2573746],Animal,(I) (2012),115 min,"\nComedy, Drama, Family",NA,NA,NA,NA,Jongwoon Choi,"[Yu-mi Kang, Namhee Kim]"
2,[tt0360481],"A Clean, Well-Lighted Place",(2002),NA,\nDrama,NA,8.3,10,NA,William Tyler Alspaugh,"[Jorge Falcão, Ana Gomes, Pedro Pulido Valente..."
3,[tt11468588],Phantom,(VI),NA,\nDrama,NA,NA,NA,NA,NA,[]
4,[tt0008852],The Atom,(1918),NA,\nDrama,NA,NA,NA,NA,Frank Borzage,"[Pauline Starke, Belle Bennett, Harry Mestayer..."
...,...,...,...,...,...,...,...,...,...,...,...
745,[tt4523360],Kosa,(2014),51 min,\nHorror,NA,2.2,22,NA,Mikhail Baranov,"[Vladislav Gavrilov, Alexandr Kartaviy, Travon..."
746,[tt1758624],Here Lies Bridget,,NA,"\nComedy, Fantasy",NA,NA,NA,NA,NA,[]
747,[tt0169983],Khabarda,(1931),61 min,\nDrama,NA,6.6,16,NA,Mikheil Chiaureli,"[S. Zavriev, P. Chkonya, Sh. Asatiani, S. Vach..."
748,[tt8790664],Bonnie,(2018),53 min,\nSci-Fi,NA,NA,NA,NA,Jacob Jean,"[Jacob Jean, Microsoft Mike]"


Il ne nous reste plus qu'à enregistrer notre dataframe en pickle et en csv :

In [ ]:
df.to_pickle("./data/df_250000_275000.pkl")

In [ ]:
df.to_csv("df_250000_275000", sep='\t', index=False)

Et on récupère l'avant dernier lien de la liste qui servira d'url de départ de la prochaine boucle (on prend bien soin de partir de l'avant dernier pour qu'entre deux bases consécutives on ait un film en commun pour faire les jointures par la suite.

In [ ]:
list_url[-2]

On procède aux jointures:

In [88]:
liste_subbase = []
for i in range(1,21):
    df_temp = pd.read_csv('subdatas/subbase' +str(i), sep = '\t', header = 0, low_memory = False)
    liste_subbase.append(df_temp)

len(liste_subbase)

20

In [93]:
df = pd.concat([subbase for subbase in liste_subbase])
len(df)
# C'est bien concatenate
df
df.reset_index()

,index,tconst,title,year,runtime,genres,metascore,rate,votes,certificate,director,casting
0,0,['tt13143964'],Borat: Subsequent Moviefilm,(2020),95 min,\nComedy,68.0,7.0,"50,191",18,Jason Woliner,"['Sacha Baron Cohen', 'Maria Bakalova', 'Tom H..."
1,1,['tt1070874'],Les Sept de Chicago,(2020),129 min,"\nDrama, History, Thriller",76.0,7.9,"37,295",16,Aaron Sorkin,"['Eddie Redmayne', 'Alex Sharp', 'Sacha Baron ..."
2,2,['tt2235695'],Rebecca,(2020),121 min,"\nDrama, Mystery, Romance",46.0,6.0,"11,829",13,Ben Wheatley,"['Lily James', 'Armie Hammer', 'Kristin Scott ..."
3,3,['tt10682266'],Hubie Halloween,(2020),102 min,"\nComedy, Fantasy, Mystery",53.0,5.2,"27,526",7,Steven Brill,"['Adam Sandler', 'Kevin James', 'Julie Bowen',..."
4,4,['tt2222042'],Love and Monsters,(2020),109 min,"\nAction, Adventure, Comedy",59.0,7.1,"10,780",PG-13,Michael Matthews,"[""Dylan O'Brien"", 'Jessica Henwick', 'Michael ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
456092,6092,['tt13366978'],Maria,(X),NaN,\nComedy,NaN,NaN,NaN,NaN,Alec Pronovost,"['Florence Longpré', 'Mariana Mazza', 'Alice P..."
456093,6093,['tt13367186'],Kill Joy,NaN,NaN,"\nAction, Crime, Drama",NaN,NaN,NaN,NaN,NaN,[]
456094,6094,['tt13367214'],Covid Plus,NaN,NaN,\nDrama,NaN,NaN,NaN,NaN,NaN,[]
456095,6095,['tt13367330'],Muralla china,(2020),NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]


In [119]:
# save de la base finale en pickle :
df.to_pickle("/Users/UPTONHENRI/Desktop/data/data.pkl")

In [16]:
# save de la base finale en csv :
df.to_csv("./Users/UPTONHENRI/Desktop/data/data.csv", sep='\t', index=False)

# Cleaning du DataFrame global

### Import de la dataframe complète

In [247]:
df = pd.read_csv("./data/data.csv", sep="\t")
df

In [249]:
pd.options.mode.chained_assignment = None 
df["year"] = df["year"].astype(str)
df["runtime"] = df["runtime"].astype(str)
df["genres"] = df["genres"].astype(str)

In [250]:
def regex(x, expr):
    expression = re.compile(expr)
    try :
        res = expression.findall(x)[0]
    except :
        res = 'NaN'
        
    return res

In [251]:
df["tconst"] = df["tconst"].apply(lambda x: regex(x, "tt\d+"))
df["year"] = df["year"].apply(lambda x : regex(x, "\d{4}"))
df["runtime"] = df["runtime"].apply(lambda x : regex(x,"\d{1,4}"))  #au cas ou plus de mille minutes !
#  df["genres"] = df["genres"].apply(lambda x : x.replace(x,"\n"))   #déjà clean en réalité

In [252]:
df.loc[df['title'] == "Forrest Gump"]

,tconst,title,year,runtime,genres,metascore,rate,votes,certificate,director,casting
181,tt0109830,Forrest Gump,1994,142,"Drama, Romance",82.0,8.8,"1,771,389",Tous publics,Robert Zemeckis,"['Tom Hanks', 'Robin Wright', 'Gary Sinise', '..."


In [253]:
df

,tconst,title,year,runtime,genres,metascore,rate,votes,certificate,director,casting
0,tt13143964,Borat: Subsequent Moviefilm,2020,95,Comedy,68.0,7.0,"50,191",18,Jason Woliner,"['Sacha Baron Cohen', 'Maria Bakalova', 'Tom H..."
1,tt1070874,Les Sept de Chicago,2020,129,"Drama, History, Thriller",76.0,7.9,"37,295",16,Aaron Sorkin,"['Eddie Redmayne', 'Alex Sharp', 'Sacha Baron ..."
2,tt2235695,Rebecca,2020,121,"Drama, Mystery, Romance",46.0,6.0,"11,829",13,Ben Wheatley,"['Lily James', 'Armie Hammer', 'Kristin Scott ..."
3,tt10682266,Hubie Halloween,2020,102,"Comedy, Fantasy, Mystery",53.0,5.2,"27,526",7,Steven Brill,"['Adam Sandler', 'Kevin James', 'Julie Bowen',..."
4,tt2222042,Love and Monsters,2020,109,"Action, Adventure, Comedy",59.0,7.1,"10,780",PG-13,Michael Matthews,"[""Dylan O'Brien"", 'Jessica Henwick', 'Michael ..."
...,...,...,...,...,...,...,...,...,...,...,...
456092,tt13366978,Maria,NaN,NaN,Comedy,NaN,NaN,NaN,NaN,Alec Pronovost,"['Florence Longpré', 'Mariana Mazza', 'Alice P..."
456093,tt13367186,Kill Joy,NaN,NaN,"Action, Crime, Drama",NaN,NaN,NaN,NaN,NaN,[]
456094,tt13367214,Covid Plus,NaN,NaN,Drama,NaN,NaN,NaN,NaN,NaN,[]
456095,tt13367330,Muralla china,2020,NaN,nan,NaN,NaN,NaN,NaN,NaN,[]


In [255]:
df_cleaned = df[df['rate'].notnull() & (df["runtime"] != "NaN") & (df["year"] != "NaN") & (df["genres"] != "nan")]

In [256]:
len(df_cleaned)

188255

In [257]:
df_cleaned.sample(10)

,tconst,title,year,runtime,genres,metascore,rate,votes,certificate,director,casting
71343,tt0296619,Les enfants de la pluie,2003,90,"Animation, Adventure, Fantasy",NaN,6.8,596,NaN,Philippe Leclerc,"['Frédéric Cerdal', 'Pascale Chemin', 'Mélody ..."
77519,tt1237932,The Truth About Angels,2011,80,"Drama, Thriller",NaN,5.3,119,NaN,Lichelli Lazar-Lea,"['Antonio Del Prete', 'Simon Rex', 'Monique Ga..."
183668,tt0126840,Cirkusrevyen 67,1967,82,"Comedy, Musical",NaN,6.9,60,NaN,Preben Kaas,"['Jytte Abildstrøm', 'Agnete Bjørn', 'Lily Bro..."
74603,tt0411733,Pandavar Bhoomi,2001,150,"Drama, Family",NaN,7.1,156,NaN,Cheran,"['Arun Vijay', 'Shamitha', 'Raj Kiran', 'Ranji..."
125108,tt0120732,Sweet Nightmare,1998,96,"Comedy, Romance",NaN,4.7,358,NaN,Fernando Fragata,"['Diogo Infante', 'Catarina Furtado', 'George ..."
80833,tt4519680,Jadoogadu,2015,151,Action,NaN,5.5,94,NaN,Yogie,"['Sonarika Bhadoria', 'Naga Shaurya', 'Ashish ..."
35136,tt1800338,Lucia de B.,2014,97,"Biography, Crime, Drama",NaN,7.1,"3,523",NaN,Paula van der Oest,"['Marwan Kenzari', 'Barry Atsma', 'Sallie Harm..."
46188,tt2746200,Sangue Azul,2014,119,"Drama, Mystery",NaN,6.4,255,NaN,Lírio Ferreira,"['Daniel de Oliveira', 'Caroline Abras', 'Sand..."
34064,tt3068192,Grace,2014,87,"Horror, Thriller",NaN,4.8,"2,285",R,Jeff Chan,"['Alexia Fast', 'Lin Shaye', 'Alan Dale', 'Ale..."
39617,tt0084718,Spring Fever,1982,100,"Comedy, Drama, Sport",NaN,4.6,121,NaN,Joseph L. Scanlan,"['Susan Anton', 'Frank Converse', 'Jessica Wal..."


### Suppression des colonnes inexploitables

In [259]:
print(df_cleaned["metascore"].isna().sum(), df_cleaned["certificate"].isna().sum())   #On oublie le metascore et le certificate

174274 158231


In [260]:
df_cleaned.drop(columns=["metascore","certificate"], inplace = True)

In [261]:
df_cleaned = df_cleaned.reset_index(drop=True)   #Réindexation
df_cleaned

,tconst,title,year,runtime,genres,rate,votes,director,casting
0,tt13143964,Borat: Subsequent Moviefilm,2020,95,Comedy,7.0,"50,191",Jason Woliner,"['Sacha Baron Cohen', 'Maria Bakalova', 'Tom H..."
1,tt1070874,Les Sept de Chicago,2020,129,"Drama, History, Thriller",7.9,"37,295",Aaron Sorkin,"['Eddie Redmayne', 'Alex Sharp', 'Sacha Baron ..."
2,tt2235695,Rebecca,2020,121,"Drama, Mystery, Romance",6.0,"11,829",Ben Wheatley,"['Lily James', 'Armie Hammer', 'Kristin Scott ..."
3,tt10682266,Hubie Halloween,2020,102,"Comedy, Fantasy, Mystery",5.2,"27,526",Steven Brill,"['Adam Sandler', 'Kevin James', 'Julie Bowen',..."
4,tt2222042,Love and Monsters,2020,109,"Action, Adventure, Comedy",7.1,"10,780",Michael Matthews,"[""Dylan O'Brien"", 'Jessica Henwick', 'Michael ..."
...,...,...,...,...,...,...,...,...,...
188250,tt13334114,Verlust,2020,110,Drama,6.5,13.0,Esmir Filho,"['Andrea Beltrão', 'Marina Lima', 'Alfredo Cas..."
188251,tt13334054,Guerra,2020,105,Drama,6.0,11.0,José Oliveira,"['Marta Ramos', 'Ana Alexandre', 'Luís Barbosa..."
188252,tt0182675,Adsiz cengaver,1970,83,"Adventure, Fantasy",5.5,124.0,Halit Refig,"['Cüneyt Arkin', 'Nebahat Çehre', 'Birsen Ayda..."
188253,tt2794316,Awara,2012,160,Action,5.5,355.0,Rabi Kinagi,"['Jeet', 'Sayantika Banerjee', 'Mukul Dev', 'A..."


### Conversion de la colonne du nombre de votes

In [263]:
df_cleaned["votes"] = df_cleaned["votes"].apply(lambda x : x.replace(',','').replace('.',''))
df_cleaned["votes"] = df_cleaned["votes"].astype("int")
df_cleaned

,tconst,title,year,runtime,genres,rate,votes,director,casting
0,tt13143964,Borat: Subsequent Moviefilm,2020,95,Comedy,7.0,50191,Jason Woliner,"['Sacha Baron Cohen', 'Maria Bakalova', 'Tom H..."
1,tt1070874,Les Sept de Chicago,2020,129,"Drama, History, Thriller",7.9,37295,Aaron Sorkin,"['Eddie Redmayne', 'Alex Sharp', 'Sacha Baron ..."
2,tt2235695,Rebecca,2020,121,"Drama, Mystery, Romance",6.0,11829,Ben Wheatley,"['Lily James', 'Armie Hammer', 'Kristin Scott ..."
3,tt10682266,Hubie Halloween,2020,102,"Comedy, Fantasy, Mystery",5.2,27526,Steven Brill,"['Adam Sandler', 'Kevin James', 'Julie Bowen',..."
4,tt2222042,Love and Monsters,2020,109,"Action, Adventure, Comedy",7.1,10780,Michael Matthews,"[""Dylan O'Brien"", 'Jessica Henwick', 'Michael ..."
...,...,...,...,...,...,...,...,...,...
188250,tt13334114,Verlust,2020,110,Drama,6.5,130,Esmir Filho,"['Andrea Beltrão', 'Marina Lima', 'Alfredo Cas..."
188251,tt13334054,Guerra,2020,105,Drama,6.0,110,José Oliveira,"['Marta Ramos', 'Ana Alexandre', 'Luís Barbosa..."
188252,tt0182675,Adsiz cengaver,1970,83,"Adventure, Fantasy",5.5,1240,Halit Refig,"['Cüneyt Arkin', 'Nebahat Çehre', 'Birsen Ayda..."
188253,tt2794316,Awara,2012,160,Action,5.5,3550,Rabi Kinagi,"['Jeet', 'Sayantika Banerjee', 'Mukul Dev', 'A..."


In [273]:
for i in [5,6]:
    print(len(df_cleaned[df_cleaned["votes"]<i]))
#bizarre mais ok car correspondait aux films pour lesquels certaines infos manquaient

0
3956


### Sauvegarde de la dataframe nettoyée

In [284]:
# save de la base nettoyée en csv :
df_cleaned.to_csv("./data/data_cleaned.csv", sep='\t', index=False)